In [ ]:
%pip install semantic-link-labs

In [ ]:
# import sempy_labs as labs
import sempy_labs as labs

# import pandas as pd
import pandas as pd

In [ ]:
# This function lists all dataflows in a specified workspace. 
# If no workspace is specified, it uses the current workspace by default.

labs.list_dataflows()

In [ ]:
# The name of the dataflow for which to find upstream dependencies.
df_name = ''

# This function looks up all the upstream dataflows for a specified dataflow.
df_lineage = labs.list_upstream_dataflows(df_name)

df_lineage

In [ ]:
import graphviz
from graphviz import Digraph
from IPython.display import display, Image

# Assuming df_lineage is your dataframe
df = df_lineage

# Create a new directed graph
dot = Digraph()

# Create a set to keep track of added edges
added_edges = set()

# Add nodes and edges based on dataframe columns
for index, row in df.iterrows():
    # Add nodes with correct labels
    dot.node(row['Dataflow Id'], label=row['Dataflow Name'], shape='cylinder')
    dot.node(row['Upstream Dataflow Id'], label=row['Upstream Dataflow Name'], shape='cylinder')
    dot.node(row['Workspace Id'], label=row['Workspace Name'], shape='folder')
    dot.node(row['Upstream Workspace Id'], label=row['Upstream Workspace Name'], shape='folder')
    
    # Add edges for dataflow dependencies
    edge = (row['Dataflow Id'], row['Upstream Dataflow Id'])
    if edge not in added_edges:
        dot.edge(*edge)
        added_edges.add(edge)
    
    # Add edges for workspace dependencies
    edge = (row['Workspace Id'], row['Upstream Workspace Id'])
    if edge not in added_edges:
        dot.edge(*edge)
        added_edges.add(edge)
    
    # Add edges to show which dataflows are in which workspaces
    edge = (row['Workspace Id'], row['Dataflow Id'])
    if edge not in added_edges:
        dot.edge(*edge, style='solid')
        added_edges.add(edge)
    
    edge = (row['Upstream Workspace Id'], row['Upstream Dataflow Id'])
    if edge not in added_edges:
        dot.edge(*edge, style='solid')
        added_edges.add(edge)

# Render the graph to a file and display it in the notebook
output_path = 'dependency_graph'
dot.render(output_path, format='png')
display(Image(filename=f'{output_path}.png'))